# Importing Libraries

In [3]:
# import libraries
!pip install texthero
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import string
    import re
    from time import time
    import texthero as hero
    from texthero import preprocessing
    from gensim.models import Word2Vec
    import nltk
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer
    from sklearn.preprocessing import LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, classification_report, ConfusionMatrixDisplay
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    from sklearn.pipeline import Pipeline
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import StackingClassifier
    from sklearn.svm import SVC
    import warnings
except(ImportError):
    print(f'Import Error: {ImportError}')

# ignore warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# set seeds for reproducability
from numpy.random import seed
seed(500)

# global configurations
pd.set_option("display.max_colwidth", -1) # show larger text in pandas dataframe

     |████████████████████████████████| 245kB 12.3MB/s 
     |████████████████████████████████| 1.4MB 31.0MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434676 sha256=78a94ac7c3682794c5eb203bb8ef576183f67f0725b409e38dffe55f42eb92e1
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
'''
- select features
- drop missing values
- combine selected features into one
'''
def data(df):
    new_df = df[['EventDescription', 'FailedAssets', 'IncidentCause', 'IncidentConsequence', 'IncidentType', 'Status', 'WeatherStation', 'Category']]
    new_df.dropna(axis=0, inplace=True)
    features = new_df['IncidentType'] + ' ' + new_df['Status'] + ' ' + new_df['WeatherStation'] + new_df['EventDescription'] + new_df['FailedAssets'] + new_df['IncidentCause'] + new_df['IncidentConsequence']
    target = new_df['Category']
    return features, target

In [5]:
# read csv
dataset = pd.read_csv("cleaned_incidents1.csv")

# get features and target
features, target = data(dataset)

# Text Cleaning and Pre-processing
def preprocess_text():
    # cleaning steps
    cleaning_pipeline = [
        preprocessing.fillna,
        preprocessing.lowercase,
        preprocessing.remove_whitespace,
        preprocessing.remove_punctuation,
        preprocessing.remove_urls,
        preprocessing.remove_brackets,
        preprocessing.remove_stopwords,
        preprocessing.remove_digits,
        preprocessing.remove_angle_brackets,
        preprocessing.remove_curly_brackets,
        preprocessing.stem
        #preprocessing.tokenize
    ]

    # apply pipeline to text
    clean_text = features.pipe(hero.clean, cleaning_pipeline)

    return clean_text

In [6]:
# check processed text
clean_text = preprocess_text()
clean_text

0       infrastructur network base report avalon airporta nearbi custom report spark electr line locat attend crew found high voltag abc conductor fault midspan result ground fire approx sqm report injuri conductor abc hvabc cabl fault midspangrassfir                                                                                                                                                                                                                               
1       infrastructur network base report laverton raafa contractor report contact earth cabl excav trench locat arriv crew found contractor denni jame ph dig trench contact earth cabl caus damag cabl report injuri third parti properti damag fail assetcontractor contact earth conductorno go zone contact                                                                                                                                                                          
2       infrastructur network base report warrnamb

In [7]:
# train test split
x_train, x_test, y_train, y_test = train_test_split(clean_text, target, random_state=0, test_size=0.25, shuffle=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4866,), (1622,), (4866,), (1622,))

In [8]:
# feature extraction methods
# tfidf
def tfidf():
    vectorizer = TfidfVectorizer(analyzer='word', max_features=1000)
    vectorizer.fit(clean_text)
    train_tfidf = vectorizer.transform(x_train)
    test_tfidf = vectorizer.transform(x_test)
    return train_tfidf, test_tfidf

# bow
def bow():
    count_vectorizer = CountVectorizer(analyzer='word', max_features=1000)
    count_vectorizer.fit(clean_text)
    train_bow = count_vectorizer.transform(x_train)
    test_bow = count_vectorizer.transform(x_test)
    return train_bow, test_bow

# bigrams
def bigrams():
    bigram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2,2), max_features=1000)
    bigram_vectorizer.fit(clean_text)
    train_bigram = bigram_vectorizer.transform(x_train)
    test_bigram = bigram_vectorizer.transform(x_test)
    return train_bigram, test_bigram

In [9]:
# get features
train_tfidf, test_tfidf = tfidf()
train_tfidf.shape, test_tfidf.shape

train_bigram, test_bigram = bigrams()

train_bow, test_bow = bow()

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier(learning_rate=0.5, verbosity=2, random_state=4)
clf.fit(train_tfidf, y_train)

In [11]:
# Model evaluation
prediction = clf.predict(test_tfidf)

acc = accuracy_score(y_test, prediction).round(4)
print("Accuracy using TF-IDF is: {}%".format(acc * 100.0))

Accuracy using TF-IDF is: 80.89%


In [30]:
# grid search
param_grid = {
    'learning_rate': [0.10, 0.30, 0.50],
    'max_depth' : [10, 20, 30],
    'min_child_weight' :[3, 5],
    'gamma' : [0.1, 0.2],
    # 'colsample_bytree' : [0.3, 0.4, 0.5, 0.7]
}
cv_xgb = GridSearchCV(estimator=XGBClassifier(random_state=4),
                      param_grid=param_grid, cv= 5, n_jobs=-1)
cv_xgb.fit(train_tfidf, y_train)
cv_xgb.best_params_

{'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 30, 'min_child_weight': 3}

In [ ]:
#Best Parameters = gamma:0.1, learning_rate_0.1, max_depth: 30, min_child_weight: 30

In [ ]:
# clf1 = XGBClassifier(gamma=0.1, learning_rate=0.1, max_depth=30, min_child_weight=3, random_state=4)
clf = XGBClassifier(gamma=0.4, learning_rate=0.05, max_depth=30, verbosity=2, random_state=4)
clf.fit(train_tfidf, y_train)

In [27]:
# Model evaluation
prediction = clf.predict(test_tfidf)

acc = accuracy_score(y_test, prediction).round(4)
print("Accuracy using TF-IDF is: {}%".format(acc * 100.0))

Accuracy using TF-IDF is: 81.44%
